In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
from torchsummary import summary
from torchvision import models

In [3]:
model = models.vgg16(pretrained=False)

In [4]:
model.classifier = nn.Sequential(nn.Linear(in_features=25088, out_features=4096, bias=True),
              nn.ReLU(inplace=True),
              nn.Dropout(p=0.5, inplace=False),
              nn.Linear(in_features=4096, out_features=4096, bias=True),
              nn.ReLU(inplace=True),
              nn.Dropout(p=0.5, inplace=False),
              nn.Linear(in_features=4096, out_features=99, bias=True)
             )

In [5]:
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from utils import get_param_size
from draft_data import input_data
from time import time

In [6]:
device = torch.device("cpu" )
model.to(device)
print("no of parameters: ", (get_param_size(model)/1000000, " Million"))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) 

no of parameters:  (134.666147, ' Million')


In [7]:
input_train = input_data(root_dir = "/media/data_dump/hemant/data/image_net/val/", type = "train")
train_dl =  DataLoader(input_train, batch_size=12,shuffle=True, num_workers=4)

input_valid = input_data(root_dir = "/media/data_dump/hemant/data/image_net/val/", type = "valid")
valid_dl =  DataLoader(input_valid, batch_size=18,shuffle=False, num_workers=4)

total number of classes are :train 99
size of data 9772
total number of classes are :valid 99
size of data 9772


In [ ]:
for j in range(1):
    model.train()
    running_loss = 0
    m = 0
    optimizer.zero_grad()
    for i, data in enumerate(train_dl, 0):

        input, target, img_name, number_of_class = data

        input, target = (input.type(torch.float32)).to(device), target.to(device)

        output = model(input)

        loss = criterion(output, target)
        running_loss += loss.item()
        
        loss.backward()

        if i % 50 == 49: # print every 50 mini-batches	
            print('[%d, %5d] dumb_loss: %.3f' %(j + 1, i + 1, running_loss ))
            running_loss = 0

        optimizer.step()
        optimizer.zero_grad()

In [ ]:
len(valid_dl)

In [ ]:
def vall():
    accuracy = 0
    with torch.no_grad():
        num = 0
        length = 0
        start = time()
        for i, data in enumerate(valid_dl, 0):	
            model.eval()
            input, target, img_name, number_of_class = data
            input= (input.type(torch.float32)).to(device)

            output = model(input)
            out , predicted = torch.max(output, 1)

            if i % 25 == 24: # print every 25 mini-batches	
                print(i)

            for i in range(len(target)):
                if target[i] == predicted[i].item():
                    num = num + 1
            length = length + len(target)
        accuracy = (num/length)*100
        end = time()
    print("accuracy is : ",accuracy, " AND ", " It took : ", (end - start), " seconds ")
torch.save(model.state_dict(),"/home/hemant/nips/model_layers"+"_" + str(accuracy) + "_nips.pth")

In [ ]:
vall()

In [ ]:
a = nn.Sequential(conv_layer(256, 256, kernel_size = 3, stride = (1,1), padding = 1),nn.Dropout(0.5))
b = []
b.append(("1",a))

In [ ]:
a

In [ ]:


from utils import conv_layer, get_param_size